### Konstantina Andronikou

## This Notebook implements a comparison between Active and Passive

In [1]:
from allennlp_models.pretrained import load_predictor

In [2]:
import checklist
import logging
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
import csv

In [3]:
from checklist.pred_wrapper import PredictorWrapper

## Active

In [4]:
#removing warnings
logging.getLogger('allennlp.common.params').disabled = True 
logging.getLogger('allennlp.nn.initializers').disabled = True 
logging.getLogger('allennlp.modules.token_embedders.embedding').disabled = True 
logging.getLogger('urllib3.connectionpool').disabled = True 
logging.getLogger('allennlp.common.plugins').disabled = True 
logging.getLogger('allennlp.common.model_card').disabled = True 
logging.getLogger('allennlp.models.archival').disabled = True 
logging.getLogger('allennlp.data.vocabulary').disabled = True 
logging.getLogger('cached_path').disabled = True

srl_predictor = load_predictor('structured-prediction-srl')
output = srl_predictor.predict("Someone drove Mary to school")
#output

In [5]:
def predict_srl(data):
    
    pred = []
    for d in data:
        pred.append(srl_predictor.predict(d))
    return pred

predict_and_conf = PredictorWrapper.wrap_predict(predict_srl)

In [6]:
d = ["Someone drove Mary to school"]
pred = predict_and_conf(d)

In [7]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][0]['description']

In [8]:
def get_arg(pred, arg_target='ARG0'):
    predicate_arguments = pred['verbs'][0]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [9]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][0]['description']
        
    return predicate_structure

In [10]:
def found_arg0_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_0 = get_arg(pred, arg_target='ARG0')

    if arg_0 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg0 = Expect.single(found_arg0_people)

In [11]:
editor = Editor()

# create examples
t = editor.template('{first_name} drove John to school.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

# for k, v in t.items():
#     print(k, v)

In [12]:
with open('Dataset_BiLSTM/Active.txt', 'w') as f:
    print(t.data, file = f)

In [13]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg0_name_default_position', expect=expect_arg0)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BiLSTM/Active.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        #print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    9 (0.9%)

Example fails:
[ARGM-MNR: Kathy] [V: drove] [ARG1: John] [ARG2: to school] .
----
[ARGM-MNR: Kathy] [V: drove] [ARG1: John] [ARG2: to school] .
----
[ARGM-MNR: Kathy] [V: drove] [ARG1: John] [ARG2: to school] .
----


In [14]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
 
original_stdout = sys.stdout  
 
with open('Predictions_BiLSTM/false_sentences_Active.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 

## PASSIVE

In [15]:
def format_srl(x, pred, conf, label=None, meta=None):
    
    return pred['verbs'][1]['description'] #second verb in this case 

In [16]:
def get_arg(pred, arg_target='ARG1'):
    predicate_arguments = pred['verbs'][1]
    #print(predicate_arguments)
    words = pred['words']
    tags = predicate_arguments['tags']
    
    arg_list = []
    for t, w in zip(tags, words):
        arg = t
        if '-' in t:
            arg = t.split('-')[1]
        if arg == arg_target:
            arg_list.append(w)
    arg_set = set(arg_list)
    return arg_set

In [17]:
def format_srl(x, pred, conf, label=None, meta=None):
    results = []
    predicate_structure = pred['verbs'][1]['description'] #second verb 
        
    return predicate_structure

In [18]:
def found_arg1_people(x, pred, conf, label=None, meta=None):
    
    # people should be recognized as arg1

    people = set([meta['first_name']])
    arg_1 = get_arg(pred, arg_target='ARG1')

    if arg_1 == people:
        pass_ = True
    else:
        pass_ = False
    return pass_


expect_arg1 = Expect.single(found_arg1_people)

In [19]:
editor = Editor()

# create examples
t = editor.template('{first_name} got driven to school by John.', meta=True, nsamples=1000, remove_duplicates = True)

#print(type(t))

# for k, v in t.items():
#     print(k, v)

In [20]:
with open('Dataset_BiLSTM/Passive.txt', 'w') as f:
    print(t.data, file = f)

In [21]:
# initialize a rest object
test = MFT(**t, name = 'detect_arg1_name_default_position', expect=expect_arg1)
output = test.run(predict_and_conf)
test.summary(format_example_fn=format_srl)
i = test.results['preds']
expect_for_i = test.results['expect_results']
with open ('Output_BiLSTM/Passive.csv','w') as f:
    writer = csv.writer(f)
    for result, exp in zip(i, expect_for_i):
        
        case = result['words']
        exp = exp
        writer.writerow([case, exp])
        #print(case,exp)

Predicting 1000 examples
Test cases:      1000
Fails (rate):    117 (11.7%)

Example fails:
Charlotte got [V: driven] [ARG2: to school] [ARG0: by John] .
----
Diane got [V: driven] [ARG2: to school] [ARG0: by John] .
----
Andrea got [V: driven] [ARG2: to school] [ARG0: by John] .
----


In [22]:
# the following piece of code was adapted from https://howtodoinjava.com/examples/python-print-to-file/
import sys
 
original_stdout = sys.stdout  
 
with open('Predictions_BiLSTM/false_sentences_Passive.txt', 'a') as f:
    sys.stdout = f 
    print(test.summary(format_example_fn=format_srl), file = f)
    # Reset the standard output
    sys.stdout = original_stdout 